In [ ]:
from numpy import nan as Nan
import numpy as np
import pandas as pd
from datetime import datetime
import time
import matplotlib.pyplot as plt
%matplotlib inline
import mplfinance as mpf
from Config import *

In [ ]:
def HA(df, numTrendHours):
    trend_buffer = pd.DataFrame(columns=["open", "close", "high", "low", "HA6_open", "HA6_close", "HA6_high", "HA6_low"])
    df['HA_close']=(df['open']+ df['high']+ df['low']+df['close'])/4

#    idx = df.index.name
#    df.reset_index(inplace=True)
#    trend_buffer = pd.DataFrame(columns=["open","high","low","close","HA6_open","HA6_high","HA6_low","HA6_close"])
#    tren_str = 0
    start_iter = 0
    for i in range(start_iter, len(df)):
        if i == start_iter:
            df.at[i, 'HA_open'] = ((df.at[i, 'open'] + df.at[i, 'close']) / 2)
        else:
            df.at[i, 'HA_open'] = ((df.at[i-1, 'HA_open'] + df.at[i-1, 'HA_close']) / 2)
            
            
        if i < 6*60*numTrendHours:
            df.at[i, "trend_strength"] = 0
        else:
            for j in range(0, 6):
                trend_buffer.at[j, "open"] = df["open"].iat[i-6*60*numTrendHours+j*60*numTrendHours]
                trend_buffer.at[j, "close"] = df["close"].iat[i-5*60*numTrendHours+j*60*numTrendHours]
                
                trend_buffer.at[j, "high"] = df["high"].iloc[(i-6*60*numTrendHours+j*60*numTrendHours):(i-5*60*numTrendHours+j*60*numTrendHours)].max()
                trend_buffer.at[j, "low"] = df["low"].iloc[(i-6*60*numTrendHours+j*60*numTrendHours):(i-5*60*numTrendHours+j*60*numTrendHours)].min()
            for j in range(0, 6):
                trend_buffer['HA6_close']=(trend_buffer['open'] + trend_buffer['high'] + trend_buffer['low'] + trend_buffer['close'])/4
                if j == 0:
                    trend_buffer.at[j, 'HA6_open'] = (trend_buffer.at[j, 'open'] + trend_buffer.at[j, 'close']) / 2
                else:
                    trend_buffer.at[j, 'HA6_open'] = (trend_buffer.at[j-1, 'HA6_open'] + trend_buffer.at[j-1, 'HA6_close']) / 2
                trend_buffer['HA6_high'] = trend_buffer[['HA6_open','HA6_close','high']].max(axis=1)
                trend_buffer['HA6_low'] = trend_buffer[['HA6_open','HA6_close','low']].min(axis=1)
            
            tren_str = trend_buffer.at[5, "HA6_close"] - trend_buffer.at[5 ,"HA6_open"]
            tren_str = tren_str/trend_buffer.at[5, "HA6_open"]
            df.at[i, "trend_strength"] = 100*tren_str
    
                
#    if idx:
#        df.set_index(idx, inplace=True)

    df['HA_high'] = df[['HA_open','HA_close','high']].max(axis=1)
    df['HA_low'] = df[['HA_open','HA_close','low']].min(axis=1)
    df["HA_difference"] = df["HA_close"] - df["HA_open"]
    df["HA_relative"] = 100*df["HA_difference"]/df["open"] 
    df["EMA200"] = df["close"].rolling(window=200).mean()
    df["EMA200"] = 100*(df["close"] - df["EMA200"])/df["close"]
    df["Level_vycnievania"] = df["HA_relative"].abs().rolling(window=10).max() 
    
    #HA trend size candle on longer time frame
        
    return df



In [ ]:
#Calculate strength
zaciatok = time.time()
data = pd.read_csv("D:\Trade_data\BTCUSD.csv")
df = HA(data, 6)
time.time() - zaciatok
df.to_csv("D:\Trade_data\LTCUSDT_recalculated.csv", index=True)

In [ ]:
#Read CSV into a Dataframe
path = "‪D:\Trade_data\BTCUSD_recalculated.csv"
path = path.strip("‪u202a")
df = pd.read_csv(path)

In [ ]:
hladina_trend = hladina_trend
hladina_break = hladina_trend
hladina_vycnievania = 5
hladina_agresivity = 0
longs = []
shorts = []
open_value_long = np.NaN
open_value_short = np.NaN
isLong = False
isShort = False
SSL = 2

Hlv0 = 1
Hlv1 = 1
df["SMA_high"] = df["HA_high"].ewm(span=SSL).mean()
df["SMA_low"] = df["HA_low"].ewm(span=SSL).mean()

for i in range(419071, len(df)):
    #SSL computation
    if df.at[i, "close"] > df.at[i,"SMA_high"]:
        Hlv0 = 1
    elif df.at[i, "close"] < df.at[i,"SMA_low"]:
        Hlv0 = -1
    else:
        Hlv0 = Hlv1
    if Hlv0 < 0:
        df.at[i, "SSL_down"] = df.at[i, "SMA_high"]
        df.at[i, "SSL_up"] = df.at[i, "SMA_low"]
        Hlv1 = Hlv0        
    else:
        df.at[i, "SSL_down"] = df.at[i, "SMA_low"]
        df.at[i, "SSL_up"] = df.at[i, "SMA_high"]
        Hlv1 = Hlv0
    if  df.at[i, "SSL_up"] > df.at[i, "SSL_down"]:
        df.at[i, "signal"] = 1.0
    else:
        df.at[i, "signal"] = 0.0
    
    #Strong Trend from 6hourHA length
    if df.at[i, "trend_strength"] > hladina_trend: # and df.at[i, "close"] < df.at[i, "EMA200"]
        isLongTrend = True
        isShortTrend = False
    elif df.at[i, "trend_strength"] < -1*hladina_trend: # and df.at[i, "close"] > df.at[i, "EMA200"]
        isShortTrend = True
        isLongTrend = False
    else:
        isLongTrend = False
        isShortTrend = False
    #Breakout
    if ((df.at[i-1, "HA_relative"] > hladina_break) and
        (df.at[i, "HA_close"] > df.at[i-1, "HA_close"]) and
        (df.at[i, "HA_high"] > df.at[i-1, "HA_high"])):
            isLongBreak = True
    elif ((df.at[i-1, "HA_relative"] < -1*hladina_break) and 
        (df.at[i, "HA_close"] < df.at[i-1, "HA_close"]) and
        (df.at[i, "HA_low"] < df.at[i-1, "HA_low"])):
            isShortBreak = True
    else:
        isLongBreak = False
        isShortBreak = False
        
    #Vycnieva
    if df.at[i-3, "Level_vycnievania"] < hladina_vycnievania:
        nevycnieva = True
    else:
        nevycnieva = False

    #Kontrola vstupu do tradu
    if isLong == False:
        if isLongTrend and isLongBreak and nevycnieva:# and isAgressiveLong:        
            start_time_long = df.at[i, "start_at"]
            ls = "long"
            open_value_long = df.at[i, "close"]
            isLong = True
    if isShort == False:
        if isShortTrend and isShortBreak and nevycnieva:# and isAgressiveShort:
            start_time_short = df.at[i, "start_at"]
            ls = "short"
            open_value_short = df.at[i, "close"]
            isShort = True
        
    #Kontrola výstupu z tradu
    if isLong:
        if ((df.at[i, "signal"] - df.at[i-1, "signal"]) == -1) or ((df.at[i, "close"] - open_value_long)/open_value_long)*100 < -1:
            close_value_long = df.at[i, "open"]
            close_time_long = df.at[i, "start_at"]
            long = [start_time_long, close_time_long, "Long", open_value_long, close_value_long]
            longs.append(long)
            isLong = False

            
    if isShort:
        if ((df.at[i, "signal"] - df.at[i-1, "signal"]) == 1) or ((df.at[i, "close"] - open_value_short)/open_value_short)*100 > 1:
            close_value_short = df.at[i, "open"]
            close_time_short = df.at[i, "start_at"]
            short = [start_time_short, close_time_short, "Short", open_value_short, close_value_short]
            shorts.append(short)
            isShort = False

        
longs = pd.DataFrame(longs, columns=["open_time", "close_time", "type", "open_value", "close_value"])
shorts = pd.DataFrame(shorts, columns=["open_time", "close_time", "type", "open_value", "close_value"])
longs["pl"] = (longs["close_value"] - longs["open_value"])/longs["open_value"]*100 - 0.15
shorts["pl"] = (shorts["open_value"] - shorts["close_value"])/shorts["open_value"]*100 - 0.15
trades = pd.concat([longs, shorts])
trades = trades.sort_values(by=["open_time"], ignore_index=True)
fitness = round(trades["pl"].sum(), 2)
trades["cum_pl"] = trades["pl"].cumsum()
wins = trades.where(trades["pl"] > 0).dropna().reset_index(drop=True)
loses = trades.where(trades["pl"] < 0).dropna().reset_index(drop=True)
wl_ratio = round(len(wins.index)/len(trades.index)*100, 2)

trades["tf"] = trades["pl"]/100
for i in range(len(trades)):
    if equity < 100000000:
        trades.at[i, "lot"] = equity*risk
        trades.at[i, "real_profit"] = trades.at[i, "tf"]*trades.at[i, "lot"]*leverage
        equity = equity + trades.at[i, "real_profit"]
    else:
        trades.at[i, "lot"] = 10000*risk
        trades.at[i, "real_profit"] = trades.at[i, "tf"]*trades.at[i, "lot"]*leverage
        equity = equity + trades.at[i, "real_profit"]

trades["cum_sum"] = trades["real_profit"].cumsum()

                
print("================================================")
print("Hladina trendu: " + str(hladina_trend))
print("Hladina breakoutu: " + str(hladina_break))
print("SSL: " + str(SSL))
print("________________________________________________")    
print("Number of trades: " + str(len(trades)))
print("W/L ratio: " + str(wl_ratio))
print("Total percentual gain: " + str(fitness))
print("Average profit: " + str(round(fitness/len(trades), 2)))
print("Total profit: " + str(round(trades["cum_sum"].iat[-1], 1)))


In [ ]:
trades["time_of_trade"] = (trades["close_time"] - trades["open_time"])/60
trades

In [ ]:
# Chaikin indicator
df["N"] = ((df["close"] - df["low"]) - (df["high"] - df["close"]))/(df["high"] - df["low"])
df["M"] = df["N"]*df["volume"]
df["volume_sum"] = df["volume"].rolling(window=cajkin).sum()
df["M_sum"] = df["M"].rolling(window=cajkin).sum()
df["CMF"] = df["M_sum"]/df["volume_sum"]
#Aroon Up/Down indicator
aroon = 14
df['aroon_up'] = 100 * df["high"].rolling(aroon + 1).apply(lambda x: x.argmax()) / aroon
df['aroon_dn'] = 100 * df["low"].rolling(aroon + 1).apply(lambda x: x.argmin()) / aroon
df["AO"] = df['aroon_up'] - df['aroon_dn']

In [ ]:
#Stochastic RSI indicator
def computeRSI (data, time_window):
    diff = data.diff(1).dropna()        # diff in one field(one day)

    #this preservers dimensions off diff values
    up_chg = 0 * diff
    down_chg = 0 * diff
    
    # up change is equal to the positive difference, otherwise equal to zero
    up_chg[diff > 0] = diff[ diff>0 ]
    
    # down change is equal to negative deifference, otherwise equal to zero
    down_chg[diff < 0] = diff[ diff < 0 ]
    
    # check pandas documentation for ewm
    # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html
    # values are related to exponential decay
    # we set com=time_window-1 so we get decay alpha=1/time_window
    up_chg_avg   = up_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    down_chg_avg = down_chg.ewm(com=time_window-1 , min_periods=time_window).mean()
    
    rs = abs(up_chg_avg/down_chg_avg)
    rsi = 100 - 100/(1+rs)
    return rsi

#df['RSI'] = computeRSI(df['Adj Close'], 14)

def stochastic(data, k_window, d_window, window):
    
    # input to function is one column from df
    # containing closing price or whatever value we want to extract K and D from
    
    min_val  = data.rolling(window=window, center=False).min()
    max_val = data.rolling(window=window, center=False).max()
    
    stoch = ( (data - min_val) / (max_val - min_val) ) * 100
    
    K = stoch.rolling(window=k_window, center=False).mean() 
    #K = stoch
    
    D = K.rolling(window=d_window, center=False).mean() 


    return K, D
        
df['RSI'] = computeRSI(df['close'], 14)
df['K'], df['D'] = stochastic(df['RSI'], 3, 3, 14)

In [ ]:
print(wins["pl"].mean())
print(wins["pl"].std())
print(wins["pl"].median())

In [ ]:
print(loses["pl"].mean())
print(loses["pl"].std())
print(loses["pl"].median())

In [ ]:
wins.boxplot(column="pl")

In [ ]:
loses.boxplot(column="pl")

In [ ]:
trades.boxplot(column="pl")

In [ ]:
profit = trades.iloc[:, 10]
plt.plot(profit)